In [10]:
# Import sqlalchemy
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
import psycopg2
import os
import pandas as pd
import boto3
from io import StringIO

USERNAME = os.environ.get('RDS_USER_NAME')
PASSWORD = os.environ.get('RDS_PASSWORD')
HOSTNAME = os.environ.get('RDS_HOSTNAME')
DBNAME = os.environ.get('RDS_DBNAME')
# Create engine will create a connection between a SQLlite DB and python
engine = create_engine(f"postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOSTNAME}", echo=True)
Session = sessionmaker(bind = engine)
session = Session()
display(session)

In [11]:
s3 = boto3.resource('s3')
client = boto3.client('s3')
bucket_name = "kayak-jedha-certification-2023"
object_key = "kayak_dataset.csv"
csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
csv_string = csv_obj['Body'].read().decode('utf-8')
print(f'{object_key} has been import\n\n')
df_kayak = pd.read_csv(StringIO(csv_string))

kayak_dataset.csv has been import




In [12]:
# pushing df s3 to sql db
df_kayak.to_sql("kayak_dataset", engine)

2023-02-05 11:50:43,099 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-02-05 11:50:43,100 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-02-05 11:50:43,143 INFO sqlalchemy.engine.Engine select current_schema()
2023-02-05 11:50:43,145 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-02-05 11:50:43,160 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-02-05 11:50:43,161 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-02-05 11:50:43,188 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-02-05 11:50:43,190 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

125

In [13]:
conn = engine.connect()

with engine.begin() as conn:
    query = text("""
    SELECT city,hotel_score,hotel_adress,curr_temp,curr_weather,avg_temp, main_weather
    FROM kayak_dataset
    ORDER BY main_weather DESC, hotel_score DESC, avg_temp DESC
    """)
    # outputing sql to dataframe
    df_rds = pd.read_sql_query(query, conn)

2023-02-05 11:50:43,676 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-02-05 11:50:43,681 INFO sqlalchemy.engine.Engine 
    SELECT city,hotel_score,hotel_adress,curr_temp,curr_weather,avg_temp, main_weather
    FROM kayak_dataset
    ORDER BY main_weather DESC, hotel_score DESC, avg_temp DESC
    
2023-02-05 11:50:43,684 INFO sqlalchemy.engine.Engine [generated in 0.00299s] {}
2023-02-05 11:50:43,728 INFO sqlalchemy.engine.Engine COMMIT


In [15]:
df_rds.dropna(inplace=True)

In [16]:
df_rds

,city,hotel_score,hotel_adress,curr_temp,curr_weather,avg_temp,main_weather
9,Saintes-Maries-de-la-Mer,"9,9","20 Rue Louis Pasteur, 13460 Les Saintes-Maries...",9.87,Clear,7.90,Clear
10,Nîmes,"9,9","215 Impasse de la Clapeyrole, 30000 Nîmes, France",9.78,Clear,7.80,Clear
11,Aigues-Mortes,"9,9","40 Rue Pasteur, 30220 Aigues-Mortes, France",6.73,Clear,7.64,Clear
12,Nîmes,"9,8","Mas des Esclots Route de Générac, 30900 Nîmes,...",9.78,Clear,7.80,Clear
13,Aigues-Mortes,"9,8","Chemin Grand Carrière, 30740 Le Cailar, France",6.73,Clear,7.64,Clear
...,...,...,...,...,...,...,...
120,Nîmes,10,"2 Rue Arago, 30000 Nîmes, France",9.78,Clear,7.80,Clear
121,Aigues-Mortes,10,"3 Rue de la Marine, 30240 Le Grau-du-Roi, France",6.73,Clear,7.64,Clear
122,Aigues-Mortes,10,"22 Rue du Port, 34280 La Grande Motte, France",6.73,Clear,7.64,Clear
123,Aigues-Mortes,10,Résidence mott'land Bat. 6 jumbo. Porte 41. 4è...,6.73,Clear,7.64,Clear
